Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [42]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_selection import SelectPercentile, f_classif

In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [21]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [5]:
train['ratingCategory'].value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [45]:
cleaned = []
for i in train['description']:
    i = re.sub('[\n]', '', i)
    cleaned.append(i)
    
cleaned = pd.Series(cleaned, name='clean')
train['cleaned'] = cleaned
train.head(1)

,id,description,ratingCategory,cleaned
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,"Sometimes, when whisky is batched, a few lefto..."


In [46]:
cleaned = []
for i in test['description']:
    i = re.sub('[\n]', '', i)
    cleaned.append(i)
    
cleaned = pd.Series(cleaned, name='clean')
test['cleaned'] = cleaned
test.head(1)

,id,description,cleaned
0,3461,\nStyle: Speyside single malt scotch Color: Wa...,Style: Speyside single malt scotch Color: Waln...


In [47]:
features = 'cleaned'
target = 'ratingCategory'

X_train = train[features]
Y_train = train[target]
X_test = test[features]
#Y_test = test[target]

### Define Pipeline Components

In [48]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [59]:
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectPercentile(f_classif, percentile=10),
    PassiveAggressiveClassifier(max_iter=60, class_weight='balanced')
)

pipeline.fit(X_train, Y_train);

/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [49]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:   51.4s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [55]:
# Predictions on test sample
pred = grid_search.predict(test['cleaned'])

In [51]:
pred = pipeline.predict(test['description'])

In [56]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [57]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [30]:
subNumber = 0

In [58]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [60]:
from sklearn.decomposition import TruncatedSVD

In [61]:
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

#svd = TruncatedSVD(n_components=100, # Just here for demo. 
 #                  algorithm='randomized',
  #                 n_iter=10)

In [63]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [65]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [69]:
from sklearn.model_selection import RandomizedSearchCV

# This is the one that matters because it got the 75%

In [98]:
pipeline = make_pipeline(
    TfidfVectorizer(),
    TruncatedSVD(algorithm='randomized'),
   # SelectPercentile(),
    PassiveAggressiveClassifier(class_weight='balanced')
)


param_distributions = {
    'truncatedsvd__n_components': [10,100,250],
    #'tfidfvectorizer__max_df': (0.75, 1.0),
    'passiveaggressiveclassifier__max_iter':(10,25,50,80)
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=15, 
    cv=6, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True
)

search.fit(X_train, Y_train);

/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 12 is smaller than n_iter=15. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 6 folds for each of 12 candidates, totalling 72 fits


/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before converge

In [99]:
pipeline = search.best_estimator_

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

### Make a Submission File

In [100]:
# Predictions on test sample
#pred = grid_search.predict(test['description'])
pred = pipeline.predict(test['cleaned'])

In [101]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [102]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [103]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [104]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

#param_dist = {
    
#    'max_depth' : randint(3,10),
#    'min_samples_leaf': randint(2,15)
#}

In [105]:
# Continue Word Embedding Work Here
pipeline = make_pipeline(
    TfidfVectorizer(),
    TruncatedSVD(algorithm='randomized'),
   # SelectPercentile(),
    GradientBoostingClassifier()
)


param_distributions = {
    'truncatedsvd__n_components': [10,100,250],
    #'tfidfvectorizer__max_df': (0.75, 1.0),
    'gradientboostingclassifier__max_depth' : randint(3,10),
    'gradientboostingclassifier__min_samples_leaf': randint(2,15)
    #'passiveaggressiveclassifier__max_iter':(10,25,50,80)
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=15, 
    cv=6, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True
)

search.fit(X_train, Y_train);

Fitting 6 folds for each of 15 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

### Make a Submission File

In [ ]:
# Predictions on test sample
pipeline = search.best_estimator_
pred = pipeline.predict(test['clean'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

In [ ]:
submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [106]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [107]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [110]:
X_train = get_word_vectors(train['cleaned'])
X_test = get_word_vectors(test['cleaned'])

In [113]:
pipeline = make_pipeline(
    #TfidfVectorizer(),
    #TruncatedSVD(algorithm='randomized'),
   # SelectPercentile(),
    PassiveAggressiveClassifier(class_weight='balanced')
)


param_distributions = {
    #'truncatedsvd__n_components': [10,100,250],
    #'tfidfvectorizer__max_df': (0.75, 1.0),
    'passiveaggressiveclassifier__max_iter':(10,25,50,80)
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=15, 
    cv=6, 
    scoring='accuracy', 
    verbose=1, 
    return_train_score=True
)

search.fit(X_train, Y_train);
pipeline = search.best_estimator_

/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=15. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 6 folds for each of 4 candidates, totalling 24 fits


/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/kaitlyn/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before converge

In [116]:
pred = pipeline.predict(X_test)
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
submission.head()

,id,ratingCategory
0,3461,0
1,2604,0
2,3341,0
3,3764,0
4,2306,0


In [117]:
submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

## Section 1
* Sentiment analysis is categorizing the emotion and/or tone of a document. Usually it's just good and bad or good/neutral/bad.

* Document classification is different because it doesn't necessarily have to do with tone or emotion. For example, you could classify academic papers into different fields and it would have nothing to do with sentiment or feeling. 

* Labeled sentiment data is pretty subjective. It's actually what I used to do for a job except I didn't know that was what I was doing. I just called it qualitative analysis. I think those labels are really sentiment. They're the boiled down essence of the sentiment which is most useful. 

* It seems to me the most common application of sentiment analysis is in marketing and tracking brand opinions on social media.

## Section 2
* Based on the Google guide, I would guess it's because the "number of samples" to "numbers of words per sample" ratio is smaller